In [1]:
from itertools import product
from itertools import combinations

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import statsmodels.formula.api as smf
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

In [2]:

response = pd.read_csv("RESULTS_NSTEVENS_CumCondCount=93322_2024-01-16.csv")
response.columns=['Preview_Length', 'Match_Score', 'Tile_Size', 'Prev_Type', 'Browse_Time']

In [3]:
response.head()

,Preview_Length,Match_Score,Tile_Size,Prev_Type,Browse_Time
0,30,60,0.2,TT,17.958197
1,30,70,0.2,TT,19.052655
2,30,75,0.2,TT,18.879372
3,30,80,0.2,TT,18.489567
4,30,85,0.2,TT,18.535646


In [4]:
model = smf.ols('Browse_Time ~ Preview_Length * Match_Score', data= response).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Browse_Time   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                     621.4
Date:                Thu, 18 Jan 2024   Prob (F-statistic):               0.00
Time:                        10:56:15   Log-Likelihood:                -7606.1
No. Observations:                3600   AIC:                         1.522e+04
Df Residuals:                    3596   BIC:                         1.524e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     31.9863      0.675     47.399      0.000      30.663      33.309
Preview_Length                -0.2358      0.010    -24.585      0.000      -0.255      -0.217
Match_Score                   -0.2076      0.008    -24.939      0.000      -0.224      -0.191
Preview_Length:Match_Score     0.0033      0.000     28.484      0.000       0.003       0.003
==============================================================================
Omnibus:                      118.483   Durbin-Watson:                   0.742
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               82.999
Skew:                           0.264   Prob(JB):                     9.49e-19
Kurtosis:                       2.475   Cond. No.                     1.31e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.31e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [5]:
new_data = pd.DataFrame({'Preview_Length': [30], 'Match_Score': [90]})
model.predict(new_data)


0    15.022457
dtype: float64

In [6]:
pl = np.arange(30, 121, 1)
ms = np.arange(1, 101, 1)
X, Y = np.meshgrid(pl, ms)
input = pd.DataFrame(np.c_[X.ravel(), Y.ravel()], columns=['Preview_Length', 'Match_Score'])

input['pred_browse_time'] = model.predict(input)

In [7]:
import plotly.express as px

fig = px.scatter_3d(x=input['Preview_Length'], y=input['Match_Score'], z=input['pred_browse_time'], #labels={'x': 'X1', 'y': 'X2', 'z': 'Target Variable (y)'},
                    title='3D Scatter Plot with Plotly')
fig.show()

In [63]:
input[input['pred_browse_time'] ==input['pred_browse_time'].min()]

,Preview_Length,Match_Score,pred_browse_time
90,120,1,3.871894


In [15]:
model2 = smf.ols('Browse_Time ~ Preview_Length + np.power(Preview_Length,2) + Match_Score + np.power(Match_Score,2) + Preview_Length:Match_Score', data= response).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Browse_Time   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     2458.
Date:                Thu, 18 Jan 2024   Prob (F-statistic):               0.00
Time:                        10:59:19   Log-Likelihood:                -5683.0
No. Observations:                3600   AIC:                         1.138e+04
Df Residuals:                    3594   BIC:                         1.142e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      57.0214      0.929     61.360      0.000      55.199      58.843
Preview_Length                 -0.2605      0.007    -39.600      0.000      -0.273      -0.248
np.power(Preview_Length, 2)     0.0021   2.56e-05     81.704      0.000       0.002       0.002
Match_Score                    -0.8463      0.024    -34.583      0.000      -0.894      -0.798
np.power(Match_Score, 2)        0.0055      0.000     32.512      0.000       0.005       0.006
Preview_Length:Match_Score     -0.0002   8.59e-05     -2.818      0.005      -0.000   -7.37e-05
==============================================================================
Omnibus:                        1.450   Durbin-Watson:                   1.602
Prob(Omnibus):                  0.484   Jarque-Bera (JB):                1.434
Skew:                           0.049   Prob(JB):                        0.488
Kurtosis:                       3.006   Cond. No.                     5.54e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.54e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
input2 = pd.DataFrame(np.c_[X.ravel(), Y.ravel()], columns=['Preview_Length', 'Match_Score'])

input2['pred_browse_time'] = model2.predict(input)

In [18]:

fig = px.scatter_3d(x=input2['Preview_Length'], y=input2['Match_Score'], z=input2['pred_browse_time'], #labels={'x': 'X1', 'y': 'X2', 'z': 'Target Variable (y)'},
                    title='3D Scatter Plot with Plotly')
fig.show()

In [19]:
input[input['pred_browse_time'] ==input['pred_browse_time'].min()]

,Preview_Length,Match_Score,pred_browse_time
7135,67,79,14.990421


### Actual Data

In [ ]:
tota